# Reading List Curator — Persistent Session & State in Google ADK

A fully working, classroom-friendly project that demonstrates **persistent memory** in the Google **Agent Development Kit (ADK)**. The use-case is a **Reading List Curator**: the agent remembers your name and a structured list of articles/books across runs using a SQL database (SQLite by default).


## TL;DR

- **What this teaches:** Switching from volatile memory to **database-backed sessions** with `DatabaseSessionService`, and using **tool functions** to read/write **session state** that persists across app restarts.
- **What you get:** A runnable terminal app and an ADK-compatible agent (`root_agent`) that you can also serve via `adk web`.
- **Domain:** A **Reading List Curator** : CRUD on entries `{title, url, tags[], status, notes}` with clean, descriptive tool outputs.

## Quick demo (what you’ll see)

1. Run the app.  
2. Tell it your name.  
3. Add a few items to your reading list.  
4. Quit and run again — your list is **still there**.

## Project Structure

```
.
├─ main.py                    # App entrypoint: DB session setup + find-or-create session + terminal loop
├─ utils.py                   # Runner helper + pretty state printer + event logger
├─ memory_agent/
│  ├─ __init__.py             # Package marker
│  └─ agent.py                # Agent definition + tools (CRUD for reading list) + root_agent export
└─ (auto-created) reading_list.db  # SQLite database file (if ADK_DB_URL not provided)
```

### File-by-file (what each does)

| Path | What it contains | Why it matters |
|---|---|---|
| `main.py` | Initializes `DatabaseSessionService`, **lists or creates** a session for `(APP_NAME, USER_ID)`, constructs a `Runner`, and starts a simple terminal loop. | Shows the **session lifecycle** and how to resume state for returning users. |
| `utils.py` | `call_agent_async(...)` to stream events from the agent, `display_state(...)` to show BEFORE/AFTER snapshots, and a readable event logger. | Makes it obvious **when** state is mutated and **what** changed. |
| `memory_agent/agent.py` | The **agent** plus six **tools**: `set_user_name`, `add_item`, `list_items`, `update_item`, `annotate_item`, `remove_item`. Exports `root_agent` for ADK web discovery. | Demonstrates **tool-driven state mutation** with **descriptive JSON outputs** (no vague `"result"` key). |
| `memory_agent/__init__.py` | Empty package marker. | Allows `from memory_agent.agent import reading_agent`. |
| `reading_list.db` | SQLite DB (created on first run unless `ADK_DB_URL` overrides). | Proof of **durable state** across runs. |

## How the system starts and runs

### Big-picture architecture

```mermaid
flowchart LR
    A[User launches app] --> B[main.py]
    B --> C[DatabaseSessionService sqlite:///...]
    C --> D{Existing session for<br/>APP_NAME, USER_ID?}
    D -->|yes| E[Reuse session_id]
    D -->|no| F[Create session with INITIAL_STATE]
    E --> G[Runner agent=reading_agent]
    F --> G
    G --> H[Terminal loop: read user input]
    H --> I[runner.run_async ..., new_message]
    I --> J{Agent calls a tool?}
    J -->|yes| K[ToolContext.state read/modify/write]
    J -->|no| L[LLM response only]
    K --> M[State persisted by session service]
    L --> M
    M --> N[Final response rendered + state AFTER printed]
    N --> H
    
    style A fill:#e1f5fe
    style B fill:#f3e5f5
    style C fill:#e8f5e8
    style D fill:#fff3e0
    style E fill:#e3f2fd
    style F fill:#e3f2fd
    style G fill:#f8e8f8
    style H fill:#e8f8e8
    style I fill:#fff8e8
    style J fill:#fff3e0
    style K fill:#ffebee
    style L fill:#e8f5e8
    style M fill:#f0f4ff
    style N fill:#e8f5e8
```
**Key lesson:** The **persistence boundary** is the session service. If your tools write to `tool_context.state[...]`, those changes are **automatically persisted** by the configured `SessionService` (database-backed here).

## State model (what we store)

The session `state` is just a JSON-serializable dict. We use:

In [ ]:
# Example of the state shape (for illustration)
state = {
    "user_name": "",
    "reading_list": [
        {
            "title": "Clean Code",
            "url": "https://example.com",
            "tags": ["software", "craft"],
            "status": "queued",        # queued | reading | done
            "notes": "Recommended by Alice"
        }
    ]
}
state

### Fields at a glance

| Key | Type | Description |
|---|---|---|
| `user_name` | `str` | Optional display name for more personal responses. |
| `reading_list` | `List[dict]` | Each item has `{title, url, tags[], status, notes}`. |
| `title` | `str` | Required; becomes `"(untitled)"` if blank. |
| `url` | `str` | Optional; cleaned to an empty string if not provided. |
| `tags` | `List[str]` | Optional; normalized to a clean list of trimmed strings. |
| `status` | `str` | `queued` (default), `reading`, or `done`. |
| `notes` | `str` | Optional free text. |

## The agent and its tools

The agent (`reading_agent`) is a standard ADK agent with **clear instructions** and **six small tools**. It also exports `root_agent` so `adk web -v .` can auto-discover it.

```mermaid
flowchart TD
    subgraph Reading List Curator Agent
    direction TB
    A[LLM Instruction<br/>- greet, infer intent<br/>- choose tools<br/>- keep output concise]
    A --> T1[set_user_name]
    A --> T2[add_item]
    A --> T3[list_items]
    A --> T4[update_item]
    A --> T5[annotate_item]
    A --> T6[remove_item]
    end
```

### Tool catalog (inputs → outputs)

| Tool | Purpose | Required args | Optional args | Output keys (always descriptive) |
|---|---|---|---|---|
| `set_user_name` | Save a display name | `name:str` | – | `action, old_name, new_name, message` |
| `add_item` | Append a new reading entry | `title:str` | `url:str, tags:list[str], status:str, notes:str` | `action, item, index, message` |
| `list_items` | Return items, optionally filtered | – | `filter_status:str, filter_tag:str` | `action, count, items[], filters, message` |
| `update_item` | Edit fields by 1-based index | `index:int` | `title, url, status, notes, tags` | `action, index, before, after, message` |
| `annotate_item` | Replace notes for an item | `index:int, notes:str` | – | `action, index, old_notes, new_notes, message` |
| `remove_item` | Delete an item by index | `index:int` | – | `action, index, removed, message` |

## What actually happens on each message

### Event sequence (one turn)

```mermaid
sequenceDiagram
  participant U as User
  participant R as Runner
  participant A as Agent
  participant DB as DatabaseSessionService

  U->>R: BEFORE state snapshot (via utils.display_state) + "Add 'Clean Code' tagged software"
  R->>A: new_message
  A->>A: Parse intent & arguments
  A->>A: Choose tool: add_item
  A->>DB: Read current session.state
  A->>A: tool_context.state["reading_list"].append({...})
  A->>DB: Persist updated state (within session service)
  A-->>R: Final agent response (summarized)
  R-->>U: Text + AFTER state snapshot (via utils.display_state)
```

### Tool short-circuiting (why state is consistent)

```mermaid
flowchart LR
    X[before_tool callback<br/>implicit in ADK] -->|state loaded| Y[Run tool]
    Y -->|writes tool_context.state| Z[after_tool callback]
    Z -->|state delta recorded| DB[(SessionService)]
    DB -->|persist| DB
    
    style X fill:#e1f5fe
    style Z fill:#e1f5fe
    style DB fill:#e8f5e8
```
*(We’re not implementing custom callbacks here; ADK’s built-in flow records the state delta which the session service persists.)*

## Setup & Running

### Prerequisites
- **Python 3.10+**
- A Google API key with access to Gemini models (used by default model `gemini-2.0-flash`)
- `pip install` the ADK and dependencies (versions will depend on your environment)

### 1) Install dependencies
```bash
pip install google-adk google-genai python-dotenv
```

### 2) Configure environment
Create a `.env` in the project root (same folder as `main.py`):
```dotenv
GOOGLE_API_KEY=your_api_key_here
ADK_DB_URL=sqlite:///./reading_list.db
ADK_APP_NAME=Reading List Curator
ADK_USER_ID=demo_user
# Optional:
# AGENTOPS_API_KEY=your_agentops_key
```

### 3) Run the terminal app
```bash
python main.py
```
Try a quick script of prompts:
```
set my name to Mayank
add "Clean Code" with tag software
add "Attention Is All You Need" url https://arxiv.org/abs/1706.03762 tag nlp
list queued
update item 2 status reading
annotate item 2: key transformer paper
remove item 1
exit
```

### 4) (Optional) Run with ADK Web UI
Because `memory_agent/agent.py` exports `root_agent`, you can spin up the ADK web runner:
```bash
adk web -v .
```
Then interact with the agent in your browser. The same persisted session is used (same `(APP_NAME, USER_ID)` pair).

## Code tour (key excerpts)
### `main.py` — find or create a session

In [ ]:
DB_URL = os.getenv("ADK_DB_URL", "sqlite:///./reading_list.db")
session_service = DatabaseSessionService(db_url=DB_URL)

INITIAL_STATE = {"user_name": "", "reading_list": []}
APP_NAME = os.getenv("ADK_APP_NAME", "Reading List Curator")
USER_ID = os.getenv("ADK_USER_ID", "demo_user")

existing = session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)
if existing and existing.sessions:
    SESSION_ID = existing.sessions[0].id
else:
    SESSION_ID = session_service.create_session(
        app_name=APP_NAME, user_id=USER_ID, state=INITIAL_STATE
    ).id

runner = Runner(agent=reading_agent, app_name=APP_NAME, session_service=session_service)

### `agent.py` — example tool (add item)

In [ ]:
def add_item(title: str, url: str = "", tags: Optional[List[str]] = None,
             status: str = "queued", notes: str = "", tool_context: ToolContext = None) -> dict:
    _ensure_state(tool_context)
    item = {
        "title": title.strip() if title else "(untitled)",
        "url": (url or "").strip(),
        "tags": _normalize_tags(tags),
        "status": status if status in {"queued","reading","done"} else "queued",
        "notes": (notes or "").strip(),
    }
    rl = tool_context.state["reading_list"]
    rl.append(item)
    tool_context.state["reading_list"] = rl   # write-through

    return {"action": "add_item", "item": item, "index": len(rl),
            "message": f"Added '{item['title']}' to your reading list."}

### `utils.py` — BEFORE vs AFTER

In [ ]:
display_state(runner.session_service, runner.app_name, user_id, session_id, "State BEFORE")
async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
    await process_agent_response(event)
display_state(runner.session_service, runner.app_name, user_id, session_id, "State AFTER")

## Troubleshooting

| Symptom | Likely cause | Fix |
|---|---|---|
| `GOOGLE_API_KEY` error | Missing key | Put it in `.env` or your shell environment. |
| DB file isn’t created | Invalid `ADK_DB_URL` | Use a valid SQLAlchemy URL, e.g. `sqlite:///./reading_list.db`. |
| State never changes | Tool didn’t run / params misparsed | Check terminal logs; try explicit phrasing like `add "Book Title" tag ml`. |
| Index errors updating/removing | Using 0-based or invalid index | Tools expect **1-based** indices; list items first to confirm positions. |
| Nothing persists between runs | New `(APP_NAME, USER_ID)` each time | Keep both **constant** unless you want a separate session. |
| AgentOps not showing traces | No API key set | Add `AGENTOPS_API_KEY` to `.env` (agent already guards for missing key). |

## Extending this project
- **Swap the model:** Change `model="gemini-2.0-flash"` to another. Keep tools identical.
- **Add search:** Introduce a `search_items(query)` tool to filter by title/notes.
- **Add MCP:** Wrap a remote search (e.g., Tavily MCP) and a tool to “enrich” entries with metadata before adding them.
- **Change domain, keep flow:** Workout Logger, Habit Tracker, Bug Notes — reuse the same session logic and CRUD shape.

## Appendix: Lifecycle diagrams

### Session discovery / creation
```mermaid
flowchart LR
    subgraph Boot
    A[Load .env] --> B[Init DatabaseSessionService]
    B --> C{list_sessions APP_NAME, USER_ID}
    C -->|found| D[Reuse latest session_id]
    C -->|none| E[create_session state=INITIAL_STATE]
    D --> F[Runner...]
    E --> F
    end
    
    style C fill:#fff3e0
    style D fill:#e8f5e8
    style E fill:#e8f5e8

```mermaid
flowchart TD
    U[User text] --> P[Agent parses intent]
    P --> Q{Needs a tool?}
    Q -->|yes| T[Call tool with ToolContext]
    T --> S[tool_context.state... = new_value]
    S --> W[ADK records state delta]
    W --> DB[(DatabaseSessionService)]
    Q -->|no| R[LLM-only response]
    DB --> OUT[Final Response]
    R --> OUT
    
    style Q fill:#fff3e0
    style T fill:#e1f5fe
    style S fill:#e1f5fe
    style W fill:#e1f5fe
    style DB fill:#e8f5e8
    style OUT fill:#e8f5e8
```

## My Handles

Linkedin :- https://www.linkedin.com/in/mayank953/  
Youtube :- https://www.youtube.com/@tech.mayankagg  
Instagram :- https://www.instagram.com/tech.mayankagg/  
Substack :- https://aiwithmayank.substack.com  
Medium :- https://medium.com/@tech.mayankagg  
Udemy :- https://www.udemy.com/user/mayank-aggarwal-197/  
Github :- https://github.com/mayank953/